## Evolving NFTs

NFT with changing metadata and lineage

    [User] mints [NFT] to [Wallet]
    [NFT] default is Black
    [Wallet A] gives money to [Charity Smart Contract]
    [Job Runs] and [NFT] evolves to a different state

Matic Mainnet NFT: https://polygonscan.com/address/0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F

Items / Records Hashes: https://polygonscan.com/address/0xd988D35e1228780714b0485e06e20b3Ef884b764

Items / Metadata Merkle Root Hash: https://polygonscan.com/address/0x09F26968f52235aa18A9feDc918E3818BA1eFD67

# A. Setup (by the owner)

### 1. Create a new session as owner

In [ ]:
#!pip install web3

In [47]:
from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

import pandas as pd

from web3.auto import w3
from eth_account.messages import encode_defunct
from hexbytes import HexBytes

WEAVE_CONFIG = "config/prod_pop_lifespan.config"
nodeApi, sessionOwner = connect_weave_api(WEAVE_CONFIG)

data_collection = "evolving_nfts"
items_table = "items"
proofs_table = "compute_proofs"

CHAIN_CONFIG_NAME = "polygon"
NFT_CONTRACT = "0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F"
ROOT_HASH_CONTRACT = "0x09F26968f52235aa18A9feDc918E3818BA1eFD67"

{"res":"ok","data":"pong 1691169732753"}


### 2. Create a table to store items and their public attributes (WARNING: removes existing data)

In [52]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "pubkey": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "roles": { "type": "STRING" },
        "name": { "type": "STRING" },
        "nft_id": { "type": "STRING" },
        "metadata": { "type": "STRING" },
        "private_data": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "allowedRolesColumnIndex": 4, # Fills the column with an EdDSA signature of the record hash
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApi.dropTable(sessionOwner, data_collection, items_table).get()
reply = nodeApi.createTable(sessionOwner, data_collection, items_table, CreateOptions(False, False, layout)).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'lifespan', 'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL', 'scope': 'evolving_nfts', 'table': 'items'}}


### 3. Prepare NFTs

#### 3.1. Prepare the metadata

In [53]:
### NFT definitions

ITEMS_COUNT = 1000

items = []
for item_id in range(ITEMS_COUNT):
    nft_name = str(item_id)
    public_data = {
        "name": "Proof of Philanthropy #" + str(item_id + 1),
        "description": "Welcome to Ouroboros, please donate $1 to upgrade",
        "attributes": [ { "tier": "black" } ],
        "external_url": "https://pop.lifespan.io",
        "image": "https://pop.lifespan.io:10443/file/black.png"
    }
    private_data = {
        "tier": "black"
    }
    
    item = [
        None, # id, filled server side
        None, # timestamp, filled server side
        None, # writer, filled server side
        None, # signature, filled server side
        "*", #allowed readers
        item_id + 1,
        "nft:" + CHAIN_CONFIG_NAME + ":" + NFT_CONTRACT + ":" + str(item_id + 1),
        json.dumps(public_data),
        json.dumps(private_data)
    ]
    items.append(item)

#print(items)
data = Records(items_table, items)
res = nodeApi.write(sessionOwner, data_collection, data, WRITE_DEFAULT).get()
print(res)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'lifespan', 'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL', 'scope': 'evolving_nfts', 'table': 'items'}, 'data': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,6fMuULYhkjeK04WzVVe0AijyAxxQlPnRkyFdPBLu9r0=,64nyDLkLChosk52azxQ7FusA5N1oMg3UTAm1CkCeow9HcK2DZ82RMP8ysKbmVXFUw4B3NYSkn9s8Brp8rGUnTRow', 'ids': '2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061 2062 2063 2064 2065 2066 2067 2068 2069 2070 2071 2072 2073 2074 2075 2076 2077 2078 2079 2080 2081 2082 2083 2084 2085 2086 2087 2088 2089 2090 2091 2092 2093 2094 2095 2096 2097 2098 2099 2100 2101 2102 2103 2104 2105 2106 2107 2108 2109 2110 2111 2112 2113 2114 2115 2116 2117 2118 2119 2120 2121 2122 2123 2124 2125 2126

#### 3.2. Mint an NFT

From https://pop.lifespan.io

#### Check the NFT balance to see that the recipient received it

In [4]:
import web3

ETH_NODE = 'https://rpc-mainnet.maticvigil.com'

wallet = '0x99072461Ab46D864580c937e762473687FC597Cc' #Replace with the wallet used for minting

w3 = web3.Web3(web3.HTTPProvider(ETH_NODE))

with open("PoP_abi.json", "r") as f:
    item_abi = json.load(f)
nftContract = w3.eth.contract(abi=item_abi, address=NFT_CONTRACT)

print("NFTs for " + wallet, nftContract.functions.balanceOf(wallet).call())

NFTs for 0x99072461Ab46D864580c937e762473687FC597Cc 1


#### The metadata for these items is automatically made available via HTTPS by our node

https://pop.lifespan.io:10443/file/nft/1

### Read the NFTs metadata

In [5]:
filter = Filter(None, { "id": "ASC" }, None, [ "name" ])
reply = nodeApi.read(sessionOwner, data_collection, items_table, filter, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df.tail())

display(json.loads(df["sig"][0]))

,id,ts,pubkey,sig,roles,name,nft_id,metadata,private_data
995,996,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,996,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"
996,997,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,997,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"
997,998,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,998,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"
998,999,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,999,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"
999,1000,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,1000,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"


[{'sig': {'recordsHash': 'zxKrzg+2xZCOs9rjT2G9PtLo9LT39D+C8OCxwf7UNqM=',
   'count': '1',
   'pubKey': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL',
   'sig': '5N6fQbRQNsHRUpFy6azK6wff4cJFMSHPs9Yb6PA6LTwvT2SNtrPytuEvWwaT9VCczivu9uawifvVfonQRR91NY2c'}}]

#### Check the NFT metadata and owner

In [6]:
import hmac
import hashlib
import base58
from urllib.request import urlopen, Request
from IPython.display import Image

nft_id = 1 # Change

nftContract = w3.eth.contract(abi=item_abi, address=NFT_CONTRACT)
uri = nftContract.functions.tokenURI(int(nft_id)).call()
print("NFT item", NFT_CONTRACT)
print("NFT metadata URI", uri)

owner = nftContract.functions.ownerOf(int(nft_id)).call()
print("Owner validated:", owner == wallet)
req = Request(uri, data=None)
with urlopen(req) as f:
    public_data = json.loads(f.read().decode('utf-8'))
    print(public_data)
image = Image(url=public_data["image"], width=150, height=150)
display(image)
display(public_data)

NFT item 0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F
NFT metadata URI https://pop.lifespan.io:10443/file/nft/1
Owner validated: False
{'name': 'Proof of Philanthropy', 'description': '$1-$99 donated, nice start, keep going!', 'external_url': 'https://pop.lifespan.io', 'image': 'https://pop.lifespan.io:10443/file/bronze.jpg', 'attributes': [{'tier': 'bronze'}]}


{'name': 'Proof of Philanthropy',
 'description': '$1-$99 donated, nice start, keep going!',
 'external_url': 'https://pop.lifespan.io',
 'image': 'https://pop.lifespan.io:10443/file/bronze.jpg',
 'attributes': [{'tier': 'bronze'}]}

#### Check NFT metadata immutability proof anchoring on the blockchain

Immutability Proofs (Hashes) Smart Contract: https://polygonscan.com/address/0xd988D35e1228780714b0485e06e20b3Ef884b764

- it can take seconds to be stored on chain, depending on how busy the target blockchain is

In [45]:
reply = nodeApi.hashes(sessionOwner, data_collection, items_table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])
display(dh)

,ID,Hash
0,1,sUpyWK8UeYMfjQi4F/A+8fzG/bS6DfGe0TzkfekloR0=
1,1001,l3b4EEhF8ixDeENo4UOkDLS+VRM+C130IWTRkVEzifg=
2,1003,TdIvy492TZ+dlPkRyrrs+RPidxiF+FCoQJsdY7ndtdA=


#### Check the collection root hash on chain

Merkle Root Hash Smart Contract: https://polygonscan.com/address/0x09F26968f52235aa18A9feDc918E3818BA1eFD67

In [46]:
salt = "salt1234"
filter = Filter(None, None, None, [ "name" ], [ "name", "nft_id", "metadata", "private_data" ])
reply = nodeApi.merkleTree(sessionOwner, data_collection, items_table, filter, salt, None, READ_DEFAULT_NO_CHAIN).get()
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Root Hash", rootHash)

reply = nodeApi.rootHash(sessionOwner, data_collection, items_table).get()
#print(reply)

from web3 import Web3, HTTPProvider

ETH_NODE = "https://rpc-mainnet.maticvigil.com"

w3 = Web3(HTTPProvider(ETH_NODE))
abi = json.loads('[{"inputs": [], "stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "sender", "type": "address"}, {"indexed": false, "internalType": "uint256", "name": "id", "type": "uint256"}, {"indexed": false, "internalType": "string", "name": "hash", "type": "string"}, {"indexed": false, "internalType": "string", "name": "metadata", "type": "string"}], "name": "NewHash", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "oldOwner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "newOwner", "type": "address"}], "name": "OwnershipTransferred", "type": "event"}, {"inputs": [], "name": "Count", "outputs": [{"internalType": "uint256", "name": "", "type": "uint256"}], "stateMutability": "view", "type": "function"}, {"inputs": [], "name": "creator", "outputs": [{"internalType": "address", "name": "", "type": "address"}], "stateMutability": "view", "type": "function"}, {"inputs": [], "name": "owner", "outputs": [{"internalType": "address", "name": "", "type": "address"}], "stateMutability": "view", "type": "function"}, {"inputs": [{"internalType": "uint256", "name": "id", "type": "uint256"}], "name": "readHash", "outputs": [{"components": [{"internalType": "uint256", "name": "id", "type": "uint256"}, {"internalType": "string", "name": "hash", "type": "string"}, {"internalType": "string", "name": "metadata", "type": "string"}], "internalType": "struct WeaveHash.DataHash", "name": "", "type": "tuple"}], "stateMutability": "view", "type": "function"}, {"inputs": [], "name": "readHashes", "outputs": [{"internalType": "string[]", "name": "", "type": "string[]"}], "stateMutability": "view", "type": "function"}, {"inputs": [], "name": "resetHashes", "outputs": [], "stateMutability": "nonpayable", "type": "function"}, {"inputs": [{"internalType": "uint256", "name": "id", "type": "uint256"}, {"internalType": "string", "name": "hash", "type": "string"}, {"internalType": "string", "name": "metadata", "type": "string"}], "name": "storeHash", "outputs": [], "stateMutability": "nonpayable", "type": "function"}, {"inputs": [{"internalType": "address", "name": "newOwner", "type": "address"}], "name": "transferOwnership", "outputs": [], "stateMutability": "nonpayable", "type": "function"}]')
contract = w3.eth.contract(abi=abi, address=ROOT_HASH_CONTRACT)

data = contract.functions.readHash(0).call()

chainRootHash = data[1]
metadata = json.loads(data[2])
chainTs = metadata["ts"]
chainRootHashSignature = metadata["signature"]
print("\nRoot Hash on chain:", chainRootHash)
print("Root Hash matching:", chainRootHash == rootHash)
print("Timestamp:", chainTs)
print("Signature:", chainRootHashSignature)

toSign = chainRootHash + " " + chainTs
sigKey = nodeApi.sigKey().get()["data"]
reply = nodeApi.verifyDataSignature(sessionOwner, sigKey, chainRootHashSignature, toSign).get()
print("Check signature:", reply["data"])

Root Hash hYwi5V13TtkRdxNecHAmzbuwnRS3nN668qLBwwgpwMk

Root Hash on chain: hYwi5V13TtkRdxNecHAmzbuwnRS3nN668qLBwwgpwMk
Root Hash matching: True
Timestamp: 1690996133717
Signature: 2oaazaT6BirvUPyotQjwHfDnTwaJkSBkrerDFWCsMBxsdCKWEJqMUdHFqvo16QUTMSsKehdooq7y43tZpsmXYSZ6
Check signature: true


#### Read Compute Proofs

In [18]:
filter = Filter(None, { "id": "ASC" }, None, [ "name" ])
reply = nodeApi.read(sessionOwner, data_collection, proofs_table, filter, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df)

display(json.loads(df["sig"][0]))

,id,ts,writer,sig,data
0,86,1690995398276,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""intervalStart"":0,""sig"":{""recordsHash"":""STeZ...","{""inputHash"":""GnPRWDcDGF6kGwpgQbGxis9WQi5JuBd5..."


[{'intervalStart': 0,
  'sig': {'recordsHash': 'STeZm8WtoNKxR7Rthn1xzJ5wqZ2pOfISGJwcU/eFW8I=',
   'sig': '5wmKmdToP2tEaVsX2EGkiUv7K1JohP8vArmFvGCZ2m16eZAuB1Nk8FLt74Rtfs3Hq6wffoNcYMP9mdsGozmct3pj'}}]

## 4. Transfer some tokens

- we expect the wallet that was minted the NFT to transfer some tokens to 0xCec4B18107d8AF27fa1395315e4c002343b6a8c2
- **Note**: To be done externally from Metamask, select the account that was used for "wallet" above and send ETH, USDC, USDT, MATIC, ETH, VITA

## 5. Manually run the background task to update user NFTs

- this task is executed periodically by the node
- it updates the metadata of all NFTs based on their owner donation status

In [19]:
reply = nodeApi.compute(sessionOwner, "gcr.io/weavechain/lifespan_update", COMPUTE_DEFAULT).get()
task_reply = reply
display(reply)

{'res': 'ok',
 'target': {'operationType': 'COMPUTE',
  'organization': 'lifespan',
  'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL'},
 'data': {'inputHash': 'GnPRWDcDGF6kGwpgQbGxis9WQi5JuBd5pVwUnJGR9tdF',
  'console': '{"res":"ok","data":"pong 1690995426937"}\nUsing rates {\'ETH\': 0.0005477, \'MATIC\': 1.473, \'USDC\': 1, \'USDT\': 1, \'VITA\': 2.86}\nParsing 1000 rows\n[\'nft\', \'polygon\', \'0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F\', \'1\'] : checking transactions from 0x2bDb5DA24028bEBa806B7B0CE80D75Dd2d3412F4 to 0xCec4B18107d8AF27fa1395315e4c002343b6a8c2\n{\'jsonrpc\': \'2.0\', \'id\': 0, \'result\': {\'transfers\': [{\'blockNum\': \'0x2b81b7d\', \'uniqueId\': \'0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c:external\', \'hash\': \'0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c\', \'from\': \'0x2bdb5da24028beba806b7b0ce80d75dd2d3412f4\', \'to\': \'0xcec4b18107d8af27fa1395315e4c002343b6a8c2\', \'value\': 2, \'erc721TokenI

### Verify the NFTs private data in the table

In [20]:
filter = Filter(None, { "id": "ASC" }, None, [ "name" ])
reply = nodeApi.read(sessionOwner, data_collection, items_table, filter, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df.head())

display(json.loads(df["metadata"][0]))
display(json.loads(df["sig"][0]))

,id,ts,pubkey,sig,roles,name,nft_id,metadata,private_data
0,1001,1690994612778,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""zxKrzg+2xZCOs9rjT2G9Pt...",*,1,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0x3f8743fb12e6c755f8d3ec4d..."
1,1002,1690994613368,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""iLpUN1qj83yo/chvYXFscx...",*,2,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xf51c5c9c90d306de627f829b..."
2,1003,1690994614162,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""k+DjePof2c97fSKp3t9eYN...",*,3,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xf51c5c9c90d306de627f829b..."
3,1004,1690994614837,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""hdnm5OJlMQ3ubWDMNC8oGe...",*,4,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xcb5593532e859465c95a82b6..."
4,5,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,5,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"


{'name': 'Proof of Philanthropy',
 'description': '$1-$99 donated, nice start, keep going!',
 'external_url': 'https://pop.lifespan.io',
 'image': 'https://pop.lifespan.io:10443/file/bronze.jpg',
 'attributes': [{'tier': 'bronze'}]}

[{'sig': {'recordsHash': 'zxKrzg+2xZCOs9rjT2G9PtLo9LT39D+C8OCxwf7UNqM=',
   'count': '1',
   'pubKey': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL',
   'sig': '5N6fQbRQNsHRUpFy6azK6wff4cJFMSHPs9Yb6PA6LTwvT2SNtrPytuEvWwaT9VCczivu9uawifvVfonQRR91NY2c'}}]

#### The table with the metadata is append-only, the previous state of the NFT is still there

- note that the the old NFT metadata is retrievable for the node owner

In [21]:
reply = nodeApi.read(sessionOwner, data_collection, items_table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df.tail())

,id,ts,pubkey,sig,roles,name,nft_id,metadata,private_data
999,1000,1690994599892,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""2fiAC5pI0hePyipT05zWTn...",*,1000,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""black""}"
1000,1001,1690994612778,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""zxKrzg+2xZCOs9rjT2G9Pt...",*,1,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0x3f8743fb12e6c755f8d3ec4d..."
1001,1002,1690994613368,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""iLpUN1qj83yo/chvYXFscx...",*,2,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xf51c5c9c90d306de627f829b..."
1002,1003,1690994614162,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""k+DjePof2c97fSKp3t9eYN...",*,3,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xf51c5c9c90d306de627f829b..."
1003,1004,1690994614837,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,"[{""sig"":{""recordsHash"":""hdnm5OJlMQ3ubWDMNC8oGe...",*,4,nft:polygon:0xdF230dC0d5e4c6bBA9f1570A58bE8120...,"{""name"": ""Proof of Philanthropy"", ""description...","{""tier"": ""bronze"", ""0xcb5593532e859465c95a82b6..."


- and there are two types of immutability proofs: 
1) for the records, both old and new, all stored on blockchain https://polygonscan.com/address/0xd988D35e1228780714b0485e06e20b3Ef884b764

In [22]:
reply = nodeApi.hashes(sessionOwner, data_collection, items_table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])
display(dh)

,ID,Hash
0,1,sUpyWK8UeYMfjQi4F/A+8fzG/bS6DfGe0TzkfekloR0=
1,1001,l3b4EEhF8ixDeENo4UOkDLS+VRM+C130IWTRkVEzifg=
2,1003,TdIvy492TZ+dlPkRyrrs+RPidxiF+FCoQJsdY7ndtdA=


- and 2) there's a merkle root hash the new state of the collection, also stored on blockchain https://polygonscan.com/address/0x09F26968f52235aa18A9feDc918E3818BA1eFD67

In [23]:
reply = nodeApi.rootHash(sessionOwner, data_collection, items_table).get()
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

toSign = rootHash + " " + ts
sigKey = nodeApi.sigKey().get()["data"]
reply = nodeApi.verifyDataSignature(sessionOwner, sigKey, rootHashSignature, toSign).get()

print("Root Hash", rootHash)
print("Check signature:", reply["data"])

Root Hash hYwi5V13TtkRdxNecHAmzbuwnRS3nN668qLBwwgpwMk
Check signature: true


### The output of the task can be as verbose as needed in order to prove its actions

In [24]:
print(task_reply["data"]["console"])

{"res":"ok","data":"pong 1690995426937"}
Using rates {'ETH': 0.0005477, 'MATIC': 1.473, 'USDC': 1, 'USDT': 1, 'VITA': 2.86}
Parsing 1000 rows
['nft', 'polygon', '0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F', '1'] : checking transactions from 0x2bDb5DA24028bEBa806B7B0CE80D75Dd2d3412F4 to 0xCec4B18107d8AF27fa1395315e4c002343b6a8c2
{'jsonrpc': '2.0', 'id': 0, 'result': {'transfers': [{'blockNum': '0x2b81b7d', 'uniqueId': '0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c:external', 'hash': '0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c', 'from': '0x2bdb5da24028beba806b7b0ce80d75dd2d3412f4', 'to': '0xcec4b18107d8af27fa1395315e4c002343b6a8c2', 'value': 2, 'erc721TokenId': None, 'erc1155Metadata': None, 'tokenId': None, 'asset': 'MATIC', 'category': 'external', 'rawContract': {'value': '0x1bc16d674ec80000', 'address': None, 'decimal': '0x12'}}]}}
Using 2.0 MATIC = 1.3745704467353952 USD
Total transferred for  0x2bDb5DA24028bEBa806B7B0CE80D75Dd2d3412F4 : 1

### And there is full lineage for the docker image hash, its input and output

In [25]:
display(task_reply["data"])

{'inputHash': 'GnPRWDcDGF6kGwpgQbGxis9WQi5JuBd5pVwUnJGR9tdF',
 'console': '{"res":"ok","data":"pong 1690995426937"}\nUsing rates {\'ETH\': 0.0005477, \'MATIC\': 1.473, \'USDC\': 1, \'USDT\': 1, \'VITA\': 2.86}\nParsing 1000 rows\n[\'nft\', \'polygon\', \'0xdF230dC0d5e4c6bBA9f1570A58bE8120a33dc50F\', \'1\'] : checking transactions from 0x2bDb5DA24028bEBa806B7B0CE80D75Dd2d3412F4 to 0xCec4B18107d8AF27fa1395315e4c002343b6a8c2\n{\'jsonrpc\': \'2.0\', \'id\': 0, \'result\': {\'transfers\': [{\'blockNum\': \'0x2b81b7d\', \'uniqueId\': \'0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c:external\', \'hash\': \'0x3f8743fb12e6c755f8d3ec4d7141ff269bad9f5ab14115e0f0011f13fbdbb25c\', \'from\': \'0x2bdb5da24028beba806b7b0ce80d75dd2d3412f4\', \'to\': \'0xcec4b18107d8af27fa1395315e4c002343b6a8c2\', \'value\': 2, \'erc721TokenId\': None, \'erc1155Metadata\': None, \'tokenId\': None, \'asset\': \'MATIC\', \'category\': \'external\', \'rawContract\': {\'value\': \'0x1bc16d674ec80000\', \

- some highlights

In [26]:
print("Compute task hash:", task_reply["data"]["computeHash"])
print("Input hash:", task_reply["data"]["inputHash"])
print("Output hash:", task_reply["data"]["outputHash"])
print("Output signature:", task_reply["data"]["outputSignatureTs"])

Compute task hash: 9EKb9LnP2p1BBc2KYHVQZvtxawB6yxeoFU3HSC99rQa7
Input hash: GnPRWDcDGF6kGwpgQbGxis9WQi5JuBd5pVwUnJGR9tdF
Output hash: 5ajuwjHoLj33yG5t5UFsJtUb3vnRaJQEMPqSLz6VyoHK
Output signature: 53ggf92Pwndn3R4VufDM32qCjmVYugHpGKBg98bEAdPvdxm4Joyekv7jzRZTXuxhkbEVKQho32rGDdcqT6h4A4yJ


#### Verify the compute lineage signature

In [27]:
data = task_reply["data"]
signature = data["outputSignature"]
match = nodeApi.verifyLineageSignature(signature, data.get("inputHash"), data.get("computeHash"), data.get("paramsHash"), data["output"])
print("Signature Match:", match)

Signature Match: True


### There is full audit available for the node owner, including for reads

In [37]:
reply = nodeApi.history(sessionOwner, data_collection, items_table, None, HISTORY_DEFAULT).get()
#print(reply)
df = pd.DataFrame(reply["data"]).transpose()

display(df)
pd.DataFrame(df.iloc[-1]["history"])

,recordId,history
1,1,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
2,2,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
3,3,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
4,4,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
5,5,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
...,...,...
5120,5120,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
6037,6037,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
6038,6038,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...
6039,6039,[{'account': 'weavejZp2gbefkBnZmjmLp3Pxn8HqsWS...


,account,operation,timestamp,ip,apiKey,taskId,image
0,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690977857892999936,172.174.28.163,e2c243b3d457432083262c61d8251e34e806e726f09d4ee1,NaN,NaN
1,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690977861332000000,172.174.28.163,e2c243b3d457432083262c61d8251e34e806e726f09d4ee1,NaN,NaN
2,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690977872828000000,172.174.28.163,e2c243b3d457432083262c61d8251e34e806e726f09d4ee1,NaN,NaN
3,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690977902740000000,localhost,49d21572c345488e8ac19c25760ff439990bbfe826b556f2,5f4bca166f3743288e6865ddc34bb15b,gcr.io/weavechain/lifespan_update
4,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690978201364999936,localhost,df8b4909a42d491c8ad7f62d096246a615fe9c6347384a6b,3bbf482c7c024a1baed9f1f2a788903e,gcr.io/weavechain/lifespan_update
...,...,...,...,...,...,...,...
78,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690993201233999872,localhost,afb6da731c894107b6f503a9d0f1a9cc4ed4c81593cbcce7,a657ddc5cde54c2399d8b064154456e5,gcr.io/weavechain/lifespan_update
79,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690993501169999872,localhost,311f6e74679a4a3493738a6c7f2198ae089c76f8c002da93,fc5cb38286dd4b1e9a6cd6a35f929da4,gcr.io/weavechain/lifespan_update
80,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690993801244999936,localhost,6586c026da0b4abf8b5fdbfd01d6fd4f56bf79eb5dcf4f76,74c7a34d95d04a97beee047577efe326,gcr.io/weavechain/lifespan_update
81,weavejZp2gbefkBnZmjmLp3Pxn8HqsWSJGJXYFYStrHfW8ykL,read,1690994101156000000,localhost,3ff7f7cdb0ac42d7b2b7038796593feaec14f6201e11951c,c0de852ee9c74ecca055ebb8b8f41c43,gcr.io/weavechain/lifespan_update
